# Research Project - CUDA notebook


In [1]:
import os

token_path = f"{os.getcwd()}/../.hf_token"
with open(token_path) as f:
    token = f.read().strip()
! huggingface-cli login --token {token} --add-to-git-credential

Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/bradlet/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 7b model too much memory for my GPU in general, 2b need to load with 8bit quantization 
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

MODELS = {
    "2b": "google/gemma-2b",
    "2bi": "google/gemma-2b-it",
    "7b": "google/gemma-7b",
    "7bi": "google/gemma-7b-it",
}

MODEL = MODELS["2bi"]
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL, quantization_config=quantization_config, device_map="auto") # on GPU

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import GenerationConfig

config = {
    "max_new_tokens": 400,
    "use_cache": False,
    # "min_new_tokens": 100,
    # "no_repeat_ngram_size": 2, 
}
def generate(prompt: str) -> str:
    """
    Use in-scope tokenizer / model to generate a repsonse given the provided `prompt`.
    :returns: Decoded text output, alongside output as tokens.
    """
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda") # on GPU
    gen_config: GenerationConfig = GenerationConfig.from_dict(config)
    outputs = model.generate(**input_ids, generation_config=gen_config)
    return tokenizer.decode(outputs[0]), outputs[0]

print(generate("A full name looks like 'Bradley Thompson'. Generate a list of 50 unique male full names:")[0])

<bos>A full name looks like 'Bradley Thompson'. Generate a list of 50 male full names:

Sure, here's a list of 50 male full names:

1. Bradley Thompson
2. Ashton Carter
3. Austin Jones
4. Blake Shelton
5. Brandon Miller
6. Bryan Miller
7. Cameron Smith
8. Carter Jones
9. Chase Thompson
10. Clayton Miller
11. Connor Smith
12. Cooper Johnson
13. Dustin Miller
14. Dylan Thomas
15. Ethan Carter
16. Frederick Miller
17. Gabriel Miller
18. Harrison Miller
19. Hunter Thompson
20. Ian Stewart
21. Jacob Miller
22. James Thompson
23. Joseph Miller
24. Kevin Miller
25. Kyle Thompson
26. Landon Miller
27. Mark Thompson
28. Matthew Miller
29. Michael Miller
30. Morgan Miller
31. Nathan Miller
32. Nicholas Miller
33. Oliver Thompson
34. Owen Miller
35. Patrick Miller
36. Quinn Thompson
37. Samuel Miller
38. Scott Miller
39. Spencer Miller
40. Steven Miller
41. Thomas Miller
42. Tristan Miller
43. Tyler Thompson
44. Ulysses Thompson
45. Vincent Miller
46. William Miller
47. Wyatt Thompson
48. Zane Mi

In [8]:
import re

def get_full_names(name_category: str, count: int):
    name_gen_prompt = f"""
        A full name looks like 'Bradley Thompson' or 'Amelia Earheart'.
        A list is unique if no two elements are the same.
        Generate a unique list of {count} {name_category} full names:
    """
    text, _ = generate(name_gen_prompt)
    name_pattern = r'\d+\.\s+([A-Za-z]+\s+[A-Za-z]+)'
    # Grab each name in generated list (if generated in expected form)
    matches = re.findall(name_pattern, text)
    return matches

male_names = get_full_names("male", 50)
female_names = get_full_names("female", 50)
male_names[:10]

['Alexander Hamilton',
 'Ashton Carter',
 'Ashton Smith',
 'Ashton Williams',
 'Austin Johnson',
 'Bailey Johnson',
 'Benjamin Harrison',
 'Blake Shelton',
 'Bradley Thompson',
 'Cameron Smith']

In [10]:
PROMPT_TEMPLATE = "Assign a career, location and skills for {}, then use this information to build them a resume."

def generate_multiple_with_prompt(samples):
    """
    Run `generate` on all provided samples while formatting them into the above `PROMPT_TEMPLATE`
    """
    return [ generate(PROMPT_TEMPLATE.format(sample)) for sample in samples]

In [11]:
results = generate_multiple_with_prompt(female_names)

m_results = generate_multiple_with_prompt(male_names)

In [35]:
%%script false --no-raise-error
"""
Note: comment out the above line to run this cell
This is used to save data after generation above
"""
# Using this to save work b/c above gen can take a long time!
import pandas as pd
# pull in from `results` or `m_results`
df = pd.DataFrame([ text for text, _ in m_results ])
with open("../data/male_base_resume_gen.csv", mode="w") as f:
    f.write(df.to_csv(index=False))

In [19]:
# %%script false --no-raise-error
"""
Note: comment out the above line to run this cell
This is used to import previously generated data, instead of save it.
"""
import re
import pandas as pd

def extract(pattern, text):
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

def extract_name(text):
    return extract(r'Assign a career, location and skills for (.+?)[,.;:]', text)

def extract_career(text):
    return extract(r'\*\*Career:\*\* (.+?)\n\n', text)

def extract_skills(text):
    # Compile to enable DOTALL regex flag
    skills_pattern = re.compile(r'\*\*Skills:\*\* (.+?)\.\n\n', re.DOTALL) 
    raw_matched_skills = extract(skills_pattern, text)
    if raw_matched_skills is None:
        return None
    else:
        return [ skill.strip() for skill in raw_matched_skills.split('\n*') if skill ]
    

with open("../data/male_base_resume_gen.csv") as f:
    df = pd.read_csv(f)
df.columns = ['text'] # rename the column that was saved w/o a text name in above saving cell
df['name'] = df['text'].apply(extract_name)
df['career'] = df['text'].apply(extract_career)
df['skills'] = df['text'].apply(extract_skills)
df

,text,name,career,skills
0,"<bos>Assign a career, location and skills for ...",Alexander Hamilton,Political Thinker and Diplomat,"[Political thought, diplomacy, communication, ..."
1,"<bos>Assign a career, location and skills for ...",Ashton Carter,Social Media Manager,"[Social media marketing, content creation, aud..."
2,"<bos>Assign a career, location and skills for ...",Ashton Smith,Software Engineer,"[Programming languages (Python, Java, C++), da..."
3,"<bos>Assign a career, location and skills for ...",Ashton Williams,"Comedian, Writer, and Podcaster","[Comedy, Writing, Podcasting, Public Speaking,..."
4,"<bos>Assign a career, location and skills for ...",Austin Johnson,Comedian,None
5,"<bos>Assign a career, location and skills for ...",Bailey Johnson,Software Engineer,"[Programming languages (Python, Java, SQL), da..."
6,"<bos>Assign a career, location and skills for ...",Benjamin Harrison,Software Engineer,"[Programming languages (Java, Python, C++), da..."
7,"<bos>Assign a career, location and skills for ...",Blake Shelton,Country singer,"[Music production, songwriting, performing, re..."
8,"<bos>Assign a career, location and skills for ...",Bradley Thompson,Software Engineer,None
9,"<bos>Assign a career, location and skills for ...",Cameron Smith,Software Engineer,"[Programming languages (Python, Java, SQL), da..."


In [ ]:
import re

# Regex to pull out the Career selected in a given generated resume
pattern = r'\*\*Career:\*\*\s*(.*)'

def parse_career(sample):
    # Search for the pattern in the document
    match = re.search(pattern, sample)
    return match.group(1) if match else None

for text, _ in results:
    print(parse_career(text))

In [ ]:
import matplotlib.pyplot as plt

x = [1, 2, 3, 4]
y = [1, 4, 9, 16]

plt.figure(figsize=(5, 2.7), layout="constrained")
plt.title("Test")
plt.xlabel("run")
plt.ylabel("rise")
plt.plot(x, y, label="exponential", color="red", linewidth=3)
plt.plot(y, x, label="opposite", color="blue", linewidth=2)
plt.legend()
plt.show()